# mediapipe/docs/solutions/hand.md 분석
출처 : [mediapipi/doc/solutions/hands.md](https://github.com/google/mediapipe/blob/master/docs/solutions/hands.md#python-solution-api)

In [1]:
import inspect
from typing import TypeVar


# https://stackoverflow.com/questions/13520421/recursive-dotdict
class attrdict(dict):
    """
    a dictionary that supports dot operation
    as well as dictionary access operation
    usage: d = attrdict() or d = attrdict({'val1':'first'})
    set attributes: d.val2 = 'second' or d['val2'] = 'second'
    get attributes: d.val2 or d['val2']
    """

    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


def attr(obj) -> tuple[attrdict, attrdict, attrdict, attrdict]:
    """Tool for python object inspection. Python object has its namespace and attribute.
    Python's built-in function 'dir' is useful, but inconvenient for object namespace analysis.
    So, this function renovates python's built-in functions like dir, var, ...

    Returns:
        tuple: (obj's state_types, obj's callable_signatures, state_values, and bounded callables)
    """
    all_attr = {}
    for attribute in dir(obj):
        if not attribute.startswith("_"):
            try:
                all_attr[attribute] = getattr(obj, attribute)
            except AttributeError:
                continue

    methods = dict([(k, v) for k, v in all_attr.items() if callable(v)])

    signatures = {}
    for k, v in all_attr.items():
        if callable(v):
            try:
                signatures[k] = inspect.signature(v)  # may occur ValueError
            except ValueError:
                signatures[k] = "No signature available for built-in method"

    state_keys = sorted(list(set(all_attr.keys()) - set(methods.keys())))
    state_types = dict([(k, type(getattr(obj, k))) for k in state_keys])
    state_values = dict([(k, getattr(obj, k)) for k in state_keys])

    return attrdict(state_types), attrdict(signatures), attrdict(state_values), attrdict(methods)

In [2]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [3]:
# For static images:
IMAGE_FILES = []
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5,
) as hands:
    for idx, file in enumerate(IMAGE_FILES):
        # Read an image, flip it around y-axis for correct handedness output (see above).
        image = cv2.flip(cv2.imread(file), 1)
        # Convert the BGR image to RGB before processing.
        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Print handedness and draw hand landmarks on the image.
        print("Handedness:", results.multi_handedness)
        if not results.multi_hand_landmarks:
            continue
        image_height, image_width, _ = image.shape
        annotated_image = image.copy()
        for hand_landmarks in results.multi_hand_landmarks:
            print("hand_landmarks:", hand_landmarks)
            print(
                f"Index finger tip coordinates: (",
                f"{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, "
                f"{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})",
            )
            mp_drawing.draw_landmarks(
                annotated_image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style(),
            )
        cv2.imwrite("../images/annotated_image" + str(idx) + ".png", cv2.flip(annotated_image, 1))
        # Draw hand world landmarks.
        if not results.multi_hand_world_landmarks:
            continue
        for hand_world_landmarks in results.multi_hand_world_landmarks:
            mp_drawing.plot_landmarks(hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)

In [4]:
# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks: # 최대 2 개
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style(),
                )
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow("MediaPipe Hands", cv2.flip(image, 1))
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [5]:
ns_results = attr(results)

# results attributes type
ns_results[0]

{'multi_hand_landmarks': NoneType,
 'multi_hand_world_landmarks': NoneType,
 'multi_handedness': NoneType}

In [6]:
# results member function
ns_results[1]

{'count': <Signature (self, value, /)>,
 'index': <Signature (self, value, start=0, stop=9223372036854775807, /)>}

In [7]:
results.count?

Signature: results.count(self, value, /)
Docstring: Return number of occurrences of value.
Type:      method_descriptor

In [8]:
results.index?

Signature: results.index(self, value, start=0, stop=9223372036854775807, /)
Docstring:
Return first index of value.

Raises ValueError if the value is not present.
Type:      method_descriptor

In [9]:
len(results.multi_hand_landmarks)

TypeError: object of type 'NoneType' has no len()

![hand landmarks](https://camo.githubusercontent.com/b0f077393b25552492ef5dd7cd9fd13f386e8bb480fa4ed94ce42ede812066a1/68747470733a2f2f6d65646961706970652e6465762f696d616765732f6d6f62696c652f68616e645f6c616e646d61726b732e706e67)

In [ ]:
for hand_landmarks in results.multi_hand_landmarks:
    print(f"Hand {hand_landmarks.LANDMARK_FIELD_NUMBER}")
    for i, mark in enumerate(hand_landmarks.landmark):
        print(f"  Hand {hand_landmarks.LANDMARK_FIELD_NUMBER}.{i:>2}: {mark.x}, {mark.y}, {mark.z}")

Hand 1
  Hand 1. 0: 0.8597419261932373, 0.7375145554542542, -8.554186692322219e-09
  Hand 1. 1: 0.7651691436767578, 0.7170498371124268, -0.049574997276067734
  Hand 1. 2: 0.704384446144104, 0.6542525887489319, -0.09301326423883438
  Hand 1. 3: 0.6666319370269775, 0.5965675115585327, -0.13881364464759827
  Hand 1. 4: 0.6182904243469238, 0.5409877300262451, -0.18813316524028778
  Hand 1. 5: 0.7342226505279541, 0.5022770762443542, -0.061190810054540634
  Hand 1. 6: 0.6971369981765747, 0.4339510202407837, -0.12984658777713776
  Hand 1. 7: 0.6679109334945679, 0.4176744818687439, -0.1845065802335739
  Hand 1. 8: 0.6386588215827942, 0.4217081665992737, -0.2221987247467041
  Hand 1. 9: 0.7880780100822449, 0.4853808879852295, -0.07982461154460907
  Hand 1.10: 0.762995719909668, 0.3923148512840271, -0.163365438580513
  Hand 1.11: 0.7334391474723816, 0.358612596988678, -0.2391130030155182
  Hand 1.12: 0.7010366320610046, 0.34276285767555237, -0.2900940775871277
  Hand 1.13: 0.8458290696144104, 0.

# solutions = mediapipe.python.solutions API
```python
"""MediaPipe Solutions Python API."""

import mediapipe.python.solutions.drawing_styles
import mediapipe.python.solutions.drawing_utils
import mediapipe.python.solutions.face_detection
import mediapipe.python.solutions.face_mesh
import mediapipe.python.solutions.face_mesh_connections
import mediapipe.python.solutions.hands
import mediapipe.python.solutions.hands_connections
import mediapipe.python.solutions.holistic
import mediapipe.python.solutions.objectron
import mediapipe.python.solutions.pose
import mediapipe.python.solutions.selfie_segmentation
```

In [10]:
import mediapipe.python.solutions.drawing_styles as drawing_styles
import mediapipe.python.solutions.drawing_utils as drawing_utils
import mediapipe.python.solutions.face_detection as face_detection
import mediapipe.python.solutions.face_mesh as face_mesh
import mediapipe.python.solutions.face_mesh_connections as face_mesh_connections
import mediapipe.python.solutions.hands as hands
import mediapipe.python.solutions.hands_connections as hands_connections
import mediapipe.python.solutions.holistic as holistic
import mediapipe.python.solutions.objectron as objectron
import mediapipe.python.solutions.pose as pos
import mediapipe.python.solutions.selfie_segmentation as selfie_segmentation

In [ ]:
attr(drawing_utils)[:3]

({'BLACK_COLOR': tuple,
  'BLUE_COLOR': tuple,
  'GREEN_COLOR': tuple,
  'RED_COLOR': tuple,
  'WHITE_COLOR': tuple,
  'cv2': module,
  'dataclasses': module,
  'detection_pb2': module,
  'landmark_pb2': module,
  'location_data_pb2': module,
  'math': module,
  'np': module,
  'plt': module},
 {'DrawingSpec': <Signature (color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2) -> None>,
  'List': <Signature (*args, **kwargs)>,
  'Mapping': <Signature (*args, **kwargs)>,
  'Optional': <Signature (*args, **kwds)>,
  'Tuple': <Signature (*args, **kwargs)>,
  'Union': <Signature (*args, **kwds)>,
  'draw_axis': <Signature (image: numpy.ndarray, rotation: numpy.ndarray, translation: numpy.ndarray, focal_length: Tuple[float, float] = (1.0, 1.0), principal_point: Tuple[float, float] = (0.0, 0.0), axis_length: float = 0.1, axis_drawing_spec: mediapipe.python.solutions.drawing_utils.DrawingSpec = DrawingSpec(color=(224, 224, 224), thickness=2, circle_radius=2)

```python
mp_drawing.draw_landmarks(
    image,
    hand_landmarks,
    mp_hands.HAND_CONNECTIONS,
    mp_drawing_styles.get_default_hand_landmarks_style(),
    mp_drawing_styles.get_default_hand_connections_style(),
)
```

In [11]:
drawing_utils.draw_landmarks?

Signature:
drawing_utils.draw_landmarks(
    image: numpy.ndarray,
    landmark_list: mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList,
    connections: Optional[List[Tuple[int, int]]] = None,
    landmark_drawing_spec: Union[mediapipe.python.solutions.drawing_utils.DrawingSpec, Mapping[int, mediapipe.python.solutions.drawing_utils.DrawingSpec]] = DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
    connection_drawing_spec: Union[mediapipe.python.solutions.drawing_utils.DrawingSpec, Mapping[Tuple[int, int], mediapipe.python.solutions.drawing_utils.DrawingSpec]] = DrawingSpec(color=(224, 224, 224), thickness=2, circle_radius=2),
)
Docstring:
Draws the landmarks and the connections on the image.

Args:
  image: A three channel BGR image represented as numpy ndarray.
  landmark_list: A normalized landmark list proto message to be annotated on
    the image.
  connections: A list of landmark index tuples that specifies how landmarks to
    be connected in the dr

In [ ]:
attr(drawing_styles)[:3]

({'face_mesh_connections': module, 'hands_connections': module},
 {'DrawingSpec': <Signature (color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2) -> None>,
  'HandLandmark': <Signature (value, names=None, *, module=None, qualname=None, type=None, start=1, boundary=None)>,
  'Mapping': <Signature (*args, **kwargs)>,
  'PoseLandmark': <Signature (value, names=None, *, module=None, qualname=None, type=None, start=1, boundary=None)>,
  'Tuple': <Signature (*args, **kwargs)>,
  'get_default_face_mesh_contours_style': <Signature (i: int = 0) -> Mapping[Tuple[int, int], mediapipe.python.solutions.drawing_utils.DrawingSpec]>,
  'get_default_face_mesh_iris_connections_style': <Signature () -> Mapping[Tuple[int, int], mediapipe.python.solutions.drawing_utils.DrawingSpec]>,
  'get_default_face_mesh_tesselation_style': <Signature () -> mediapipe.python.solutions.drawing_utils.DrawingSpec>,
  'get_default_hand_connections_style': <Signature () -> Mapping[Tuple

In [ ]:
drawing_styles.get_default_hand_landmarks_style()

{<HandLandmark.WRIST: 0>: DrawingSpec(color=(48, 48, 255), thickness=-1, circle_radius=5),
 <HandLandmark.THUMB_CMC: 1>: DrawingSpec(color=(48, 48, 255), thickness=-1, circle_radius=5),
 <HandLandmark.INDEX_FINGER_MCP: 5>: DrawingSpec(color=(48, 48, 255), thickness=-1, circle_radius=5),
 <HandLandmark.MIDDLE_FINGER_MCP: 9>: DrawingSpec(color=(48, 48, 255), thickness=-1, circle_radius=5),
 <HandLandmark.RING_FINGER_MCP: 13>: DrawingSpec(color=(48, 48, 255), thickness=-1, circle_radius=5),
 <HandLandmark.PINKY_MCP: 17>: DrawingSpec(color=(48, 48, 255), thickness=-1, circle_radius=5),
 <HandLandmark.THUMB_MCP: 2>: DrawingSpec(color=(180, 229, 255), thickness=-1, circle_radius=5),
 <HandLandmark.THUMB_IP: 3>: DrawingSpec(color=(180, 229, 255), thickness=-1, circle_radius=5),
 <HandLandmark.THUMB_TIP: 4>: DrawingSpec(color=(180, 229, 255), thickness=-1, circle_radius=5),
 <HandLandmark.INDEX_FINGER_PIP: 6>: DrawingSpec(color=(128, 64, 128), thickness=-1, circle_radius=5),
 <HandLandmark.IND

#### site-packages\\mediapipe\\python\\solutions\\hands_connections.py
```python
# Copyright 2021 The MediaPipe Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""MediaPipe Hands connections."""

HAND_PALM_CONNECTIONS = ((0, 1), (0, 5), (9, 13), (13, 17), (5, 9), (0, 17))

HAND_THUMB_CONNECTIONS = ((1, 2), (2, 3), (3, 4))

HAND_INDEX_FINGER_CONNECTIONS = ((5, 6), (6, 7), (7, 8))

HAND_MIDDLE_FINGER_CONNECTIONS = ((9, 10), (10, 11), (11, 12))

HAND_RING_FINGER_CONNECTIONS = ((13, 14), (14, 15), (15, 16))

HAND_PINKY_FINGER_CONNECTIONS = ((17, 18), (18, 19), (19, 20))

HAND_CONNECTIONS = frozenset().union(*[
    HAND_PALM_CONNECTIONS, HAND_THUMB_CONNECTIONS,
    HAND_INDEX_FINGER_CONNECTIONS, HAND_MIDDLE_FINGER_CONNECTIONS,
    HAND_RING_FINGER_CONNECTIONS, HAND_PINKY_FINGER_CONNECTIONS
])
```